# **Simulation of a crack growth in a square plate based on Peridynamic Theory**

Silling, S. A., & Askari, E. (2005). *A meshfree method based on the peridynamic model of solid mechanics*. *Computers & Structures, 83*(17-18), 1526-1535. https://doi.org/10.1016/j.compstruc.2004.11.026

## Setup

### Load Python Libraries

- **NumPy:** This library is fundamental for numerical computations in Python. It provides support for arrays, matrices, and many mathematical functions that operate on these data structures.
- **Matplotlib:** A plotting library for creating static, animated, and interactive visualizations in Python. It's particularly useful for generating plots, histograms, and other types of charts.
- **Pandas:** A Python library used for working with data sets. It has functions for analyzing, cleaning, exploring, and manipulating data.
- **Seaborn:** This Python library helps users visualize data through statistical graphics.
- **SciPy (KDTree):** This module in SciPy provides efficient nearest-neighbor and range search operations in k-dimensional space.
- **Celluloid:** This Python library simplifies the process of creating animations by capturing each frame in Matplotlib.
- **IPython.display.HTML:** This module from IPython is used to display rich HTML content within Jupyter notebooks, helpful for embedding animations and media.
- **PillowWriter:** This writer class in Matplotlib saves animations in GIF format, particularly useful when working with the FuncAnimation class to create smooth animations.

In [3]:
import numpy as np  
import matplotlib.pyplot as plt 
from matplotlib.animation import FuncAnimation, PillowWriter
from matplotlib import cm
import pandas as pd  
import seaborn as sns
from scipy.spatial import KDTree
from celluloid import Camera
from IPython.display import HTML

## Parameters

<center><img src="fig4-square_plate_containing_an_initial_center_crack.gif"/></center>
<br><br> <center> Figure - Square Plate Containing an Initial Center Crack (Silling & Askari, 2005) </center>

In [50]:
#Define the dimensions of the 2D Plate
length = 0.05                #Length of the plate
width = 0.05               #Width of the plate

#Definte the number of nodes in x & y directions
nx = 50                     #Number of nodes in x-direction
ny = 50                     #Number of nodes in y-direction

#Crack
a0 = 5                      #Half length of the initial center crack 
crack_length = 2 * a0       #Total length of the initial center crack

#Other Properties  
dx = length/nx              #Spacing
num_boundary_rows = 3       #Number of boundary regions
total_nodes = nx * (ny + (num_boundary_rows*2)) #Total number of nodes in the 2D plate
delta_x = 1.6 #3.015 * dx          #Horizon
radius_of_node = dx / 2     #Radius around the node
area_of_node = dx * dx      #Area of the radius around the node
vol = area_of_node * dx     #Volume of the node
thick = dx                  #Total thickness of the plate
rho = 8000                 #Density
E = 192e9                   #Elastic Modulus
c = (9 * E ) / (np.pi * thick * delta_x**3) #Material constant
poisson_ratio = 1/3         #Poisson's ratio
critical_stretch = 0.02     #Critical stretch for bond failure

#Time
dt = np.sqrt(2 * rho * dx / (np.pi * delta_x**2 * dx * c))
total_time = 15             #Total time of simulation
delta_t = 0.012
total_time_steps = int(total_time/delta_t)

In [63]:
nodefam.shape

(1000000,)

## Create Grid Geometry

In [51]:
# Generate internal region points
x_internal = np.linspace(-length / 2 + dx / 2, length / 2 - dx / 2, nx)
y_internal = np.linspace(-width / 2 + dx / 2, width / 2 - dx / 2, ny)
X_internal, Y_internal = np.meshgrid(x_internal, y_internal)
internal_points = np.column_stack((X_internal.ravel(), Y_internal.ravel()))

# Generate boundary region points (bottom and top)
x_boundary = np.linspace(-length / 2 + dx / 2, length / 2 - dx / 2, nx)
y_bottom = np.linspace(-width / 2 - dx / 2, -width / 2 - num_boundary_rows * dx + dx / 2, num_boundary_rows)
y_top = np.linspace(width / 2 + dx / 2, width / 2 + num_boundary_rows * dx - dx / 2, num_boundary_rows)

X_bottom, Y_bottom = np.meshgrid(x_boundary, y_bottom)
X_top, Y_top = np.meshgrid(x_boundary, y_top)

bottom_points = np.column_stack((X_bottom.ravel(), Y_bottom.ravel()))
top_points = np.column_stack((X_top.ravel(), Y_top.ravel()))

# Combine all points
coord = np.zeros((total_nodes, 2))
coord = np.vstack((internal_points, bottom_points, top_points))

## Create Failure Matrix

In [60]:
#Define the maximum number of nodes that can be captured inside the horizon 
max_family_length = 100

fail_matrix = np.ones((total_nodes, max_family_length), dtype=int)

# Arrays to track the family members
pointfam = np.zeros(total_nodes, dtype=int)
numfam = np.zeros(total_nodes, dtype=int)
nodefam = np.zeros(1000000, dtype=int)

# Create a KDTree for efficient neighbor search
tree = KDTree(coord)

# Find all family members within the specified horizon distance `delta`
all_family_members = tree.query_ball_tree(tree, r=delta_x)

# Fill 'numfam', 'nodefam', and 'pointfam' arrays
current_index = 0  # Track the current position in nodefam

for i in range(total_nodes):
    family_members = np.array(all_family_members[i], dtype=int)  # Get family members' indices as numpy array
    num_members = len(family_members)  # Number of family members

    # Store the number of family members
    numfam[i] = num_members

    # Update the `pointfam` array
    pointfam[i] = current_index

    # Store family member indices in `nodefam` ensuring exact matches with original code
    nodefam[current_index:current_index + num_members] = family_members[:num_members]

    # Move the current index forward
    current_index += num_members

# Trim unused space in `nodefam` for efficiency
nodefam = nodefam[:current_index]



ValueError: could not broadcast input array from shape (2800,) into shape (400,)

In [58]:
nodefam

array([   0,    1,    2, ..., 2797, 2798, 2799])